# ANALYSER

In [7]:
import Modele
from Variables import *
import csv
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import time
import statistics

In [8]:
# Idée de test 1 : nbr de voiture total sur la route hors fin


def nbr_voiture(route, direction, traffic, duree_feu, duree_test):

    for i in range(len(route)):
        for j in range(len(route[i])):
            if route[i][j] != 0 and route[i][j][0] == "Feu":
                route[i][j][2] = duree_feu

    res = []

    for temps in range(duree_test):

        route, direction, traffic = Modele.mouvement(route, direction, traffic, temps)

        compt = 0

        for i in range(len(traffic)):
            for j in range(len(traffic[i])):
                if isinstance(traffic[i][j], list):
                    if route[i][j] != 0 and route[i][j][0] == "Route":
                        compt += traffic[i][j].count(1)
                    elif route[i][j] != 0 and route[i][j][0] != "Fin":
                        compt += traffic[i][j][0]

        res.append(compt)

    return res


def test_1(route, direction, traffic, duree_min, duree_max, nbr_test, nbr_ite):

    res = []
    
    for duree in tqdm(range(duree_min, duree_max)):
        for i in range(nbr_test):
            compt = []
            compt.append(sum(nbr_voiture(route, direction, traffic, duree, nbr_ite)))
        res.append(statistics.mean(compt))
        
    return res


r = test_1(route_etude, direction_etude, traffic_etude, 30, 60, 10, 240)
index = sorted(range(len(r)), key=lambda i: r[i])

print(r)
print([i + 30 for i in index])

100%|██████████| 30/30 [00:06<00:00,  4.85it/s]

[1474258, 1476362, 1468644, 1464551, 1442019, 1450296, 1453560, 1459681, 1462426, 1465756, 1473313, 1468478, 1484513, 1489224, 1485628, 1477476, 1459597, 1444202, 1448107, 1459273, 1464874, 1476385, 1473374, 1478535, 1490553, 1507941, 1528435, 1538740, 1545906, 1545637]
[34, 47, 48, 35, 36, 49, 46, 37, 38, 33, 50, 39, 41, 32, 40, 52, 30, 31, 51, 45, 53, 42, 44, 43, 54, 55, 56, 57, 59, 58]


Résultat : Test naif qui a pour objectif de vider au plus vite les routes sans vraiment vouloir minimiser les embouteillage. On obtient en général un temps optimal le plus faible étrange !

In [9]:
# Test 2 : Nombre de voiture sur les routes = embouteillage
# On veut que le phenomène soit penalisé x2 si plus de la moitié de la route occupé par des voitures
# Et x5 les voitures sur les cases départ avec plus de 1 voiture

def nbr_voiture_coef(route, direction, traffic, duree_feu, duree_test):

    for i in range(len(route)):
        for j in range(len(route[i])):
            if route[i][j] != 0 and route[i][j][0] == "Feu":
                route[i][j][2] = duree_feu

    res = []

    for temps in range(duree_test):

        route, direction, traffic = Modele.mouvement(route, direction, traffic, temps)

        compt = 0

        for i in range(len(traffic)):
            for j in range(len(traffic[i])):
                if isinstance(traffic[i][j], list):
                    if route[i][j] != 0 and route[i][j][0] == "Route":
                        if traffic[i][j].count(1) > len(route[i][j])/2:
                            compt += 5*traffic[i][j].count(1)
                        else:
                            compt += traffic[i][j].count(1)
                    elif route[i][j] != 0 and route[i][j][0] == "Départ":
                        if traffic[i][j][0] > 1:
                            compt += 10*traffic[i][j][0]
                        else:
                            compt += traffic[i][j][0]
                    elif route[i][j] != 0 and route[i][j][0] != "Fin":
                        compt += traffic[i][j][0]

        res.append(compt)

    return res

def test_2(route, direction, traffic, duree_min, duree_max, nbr_test, nbr_ite):

    res = []
    
    for duree in tqdm(range(duree_min, duree_max)):
        for i in range(nbr_test):
            compt = []
            compt.append(sum(nbr_voiture_coef(route, direction, traffic, duree, nbr_ite)))
        res.append(statistics.mean(compt))
        
    return res

r = test_2(route_etude, direction_etude, traffic_etude, 30, 60, 10, 240)
index = sorted(range(len(r)), key=lambda i: r[i])

print(r)
print([i + 30 for i in index])

100%|██████████| 30/30 [00:06<00:00,  4.59it/s]

[1576196, 1587232, 1584632, 1568618, 1571185, 1572300, 1579165, 1600386, 1600421, 1616216, 1616155, 1620197, 1620262, 1619465, 1620309, 1605754, 1605604, 1599261, 1607902, 1603540, 1613739, 1628002, 1632751, 1634781, 1635724, 1655080, 1658365, 1667004, 1678564, 1688628]
[33, 34, 35, 30, 36, 32, 31, 47, 37, 38, 49, 46, 45, 48, 50, 40, 39, 43, 41, 42, 44, 51, 52, 53, 54, 55, 56, 57, 58, 59]


In [1]:
r = [0,3,3]

n = len(r)

r[n]

IndexError: list index out of range